<h2>Install knowledge_base_manager package</h2>

In [14]:
! pip install -i https://test.pypi.org/simple/ knowledge-base-manager==0.1.13

Looking in indexes: https://test.pypi.org/simple/
  Attempting uninstall: knowledge-base-manager
    Found existing installation: knowledge-base-manager 0.1.12
    Uninstalling knowledge-base-manager-0.1.12:
      Successfully uninstalled knowledge-base-manager-0.1.12


<h2>Initialise Variables</h2>

In [1]:
import os 
from dotenv import load_dotenv
from knowledge_base_manager.core.qna_manager import QnAManager
from knowledge_base_manager.core.knowledge_base_manager import KnowledgeBaseManager
from knowledge_base_manager.types import Category

# TODO: delete these temp imports
from langchain_openai import AzureChatOpenAI

In [2]:

# Defines the instance of AzureChatOpenAI class
llm = AzureChatOpenAI(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_4o_NAME"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
    temperature=0,
)

In [4]:
# import 

from functions.agents.human_loop.feedback import FeedbackRetrieval

kb = FeedbackRetrieval()

/Users/keenlim/Final-Year-Project/Development/Course_v2/.venv/lib/python3.12/site-packages/knowledge_base_manager/core/database_manager.py:17: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self.client = pymongo.MongoClient(db_connection_str)


<h2>Usage</h2>

In [5]:
# Create the index
kb.create_index()

'coursegenie-qna'

In [6]:
# RESET index
kb.kb_manager.delete_index_function()
kb.create_index()

'coursegenie-qna'

In [7]:
# Similarity search test
user_query = "Who teaches SC1007?"
retrieved_docs = kb.kb_manager.similarity_search(user_query)

context_str=""

for doc in retrieved_docs:
    context_str.append(doc["content"])

# Retrieve context from kb_manager
print("Context_str:" ,context_str)


Context_str: 


In [7]:
def classify_query(context_str:str, query:str):

    # Form system prompt
    system_prompt = f"""
                        You are given a context and a query. Determine whether the context provides sufficient information to answer the query.

                        If the context is enough to answer the query, respond to the query using the context.

                        If the context is insufficient to answer the query, respond with "QUERY" only.
s
                        Context: {context_str}
                        Query: {query}
                        """

    llm_response = llm.invoke(system_prompt).content

    return llm_response

print(classify_query(context_str, user_query))

QUERY


In [8]:
if ("QUERY" in classify_query(context_str, user_query)):
    # add to admin applicaiton QnA database
    kb.qna_manager.add_unanswered_question(user_query)


In [12]:
# function to sync qna db to kb
kb.sync_qna_to_kb()

Added Updated Questions & Answers 1!


True